## Zadanie wykonane w ramach zajęć z przetwarzania danych ustrukturyzowanych, otrzymana maksymalna liczba punktów

Zadanie polegało na analizie danych za pomocą pakietu pandas analogicznie do zapytań sql. 
Pakiet danych pochodzi z serwisu https://travel.stackexchange.com/, składa się z następujących ramek danych:
 • Posts.csv.gz
 • Users.csv.gz
 • Comments.csv.gz
 • PostLinks.csv.gz
 • Votes.csv.gz
 
 Pakiet posts był za ciężki by go dodać na github, zawierał kolumny: Id, PostTypeId, AcceptedAnswerId, CreationDate, Score, ViewCount, Body,OwnerUserId, LastEditorUserId, LastEditDate, LastActivityDate, Title,Tags, AnswerCount, CommentCount, ClosedDate, ContentLicense, FavoriteCount, ParentId, LastEditorDisplayName, CommunityOwnedDate, OwnerDisplayName
 
 
 Dalsza część to wykonanie zadania według podanego szablonu

## 1. Przygotowanie danych

1. Wykonaj `import` potrzebnych pakietów (oprócz poniżej wymienionych), tak aby umieszczony w tym notebooku kod działał.

In [72]:
import sqlite3
import pandas as pd
import numpy as np

2. Wczytaj ramki danych, na których będziesz dalej pracował

**UWAGA:**
* Pliki muszą znajdować się w katalogu "travel_stackexchange_com" w tym samym katalogu co ten notebook (plik .ipynb).
* Nazwy tabel muszą być *zgodne z instrukcją zamieszczoną w treści pracy domowej*.


In [73]:
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz", compression='gzip')
Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz", compression='gzip')
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz", compression='gzip')
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv.gz", compression='gzip')
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv.gz", compression='gzip')

3. Przygotuj bazę danych wykonując poniższą komórkę.

In [74]:
# Ścieżka do pliku z bazą danych ('./' oznacza bieżący katalog, 
# czyli będzie to plik w tym samym katalogu, co ten notebook).

SCIEZKA_BAZY = './pd5_baza.db'  
with sqlite3.connect(SCIEZKA_BAZY) as conn: # połączenie do bazy danych
    # wewnątrz bloku `with` mamy dostępny obiekt połączenia, które jest automatycznie zamykane po jego opuszczeniu.
    Comments.to_sql("Comments", conn, if_exists='replace')  # jeżeli ramka danych już istnieje, to jest nadpisywana.
    Posts.to_sql("Posts", conn, if_exists='replace', index=False)
    Users.to_sql("Users", conn, if_exists='replace', index=False)
    PostLinks.to_sql("PostLinks", conn, if_exists='replace', index=False)
    Votes.to_sql("Votes", conn, if_exists='replace', index=False)

## 2. Wyniki zapytań SQL

Wykonaj zapytania sql. Poniższy kod zapisze też wynik do pliku bazy - potem można go z niej odczytać.

In [75]:
zapytanie_1 = """
SELECT STRFTIME('%Y', CreationDate) AS Year, 
       STRFTIME('%m', CreationDate) AS Month, 
       COUNT(*) AS TotalAccountsCount, 
       AVG(Reputation) AS AverageReputation
FROM Users
GROUP BY Year, Month
"""

zapytanie_2 = """
SELECT Users.DisplayName, Users.Location, Users.Reputation, 
       STRFTIME('%Y-%m-%d', Users.CreationDate) AS CreationDate,
       Answers.TotalCommentCount
FROM (
        SELECT OwnerUserId, SUM(CommentCount) AS TotalCommentCount
        FROM Posts
        WHERE PostTypeId == 2 AND OwnerUserId != ''
        GROUP BY OwnerUserId
     ) AS Answers
JOIN Users ON Users.Id == Answers.OwnerUserId
ORDER BY TotalCommentCount DESC
LIMIT 10
"""


zapytanie_3 = """
SELECT Spam.PostId, UsersPosts.PostTypeId, UsersPosts.Score, 
       UsersPosts.OwnerUserId, UsersPosts.DisplayName,
       UsersPosts.Reputation
FROM (
        SELECT PostId  
        FROM Votes
        WHERE VoteTypeId == 12
     ) AS Spam
JOIN (
        SELECT Posts.Id, Posts.OwnerUserId, Users.DisplayName, 
               Users.Reputation, Posts.PostTypeId, Posts.Score
        FROM Posts JOIN Users
        ON Posts.OwnerUserId = Users.Id
     ) AS UsersPosts 
ON Spam.PostId = UsersPosts.Id
"""


zapytanie_4 = """
SELECT Users.Id, Users.DisplayName, Users.UpVotes, Users.DownVotes, Users.Reputation,
       COUNT(*) AS DuplicatedQuestionsCount
FROM (
        SELECT Duplicated.RelatedPostId, Posts.OwnerUserId
        FROM (
                SELECT PostLinks.RelatedPostId
                FROM PostLinks
                WHERE PostLinks.LinkTypeId == 3
             ) AS Duplicated
        JOIN Posts
        ON Duplicated.RelatedPostId = Posts.Id
     ) AS DuplicatedPosts
JOIN Users ON Users.Id == DuplicatedPosts.OwnerUserId
GROUP BY Users.Id
HAVING DuplicatedQuestionsCount > 100
ORDER BY DuplicatedQuestionsCount DESC
"""

zapytanie_5 = """
SELECT QuestionsAnswers.Id,
       QuestionsAnswers.Title, 
       QuestionsAnswers.Score,
       MAX(Duplicated.Score) AS MaxScoreDuplicated,
       COUNT(*) AS DulicatesCount,
       CASE 
         WHEN QuestionsAnswers.Hour < '06' THEN 'Night'
         WHEN QuestionsAnswers.Hour < '12' THEN 'Morning'
         WHEN QuestionsAnswers.Hour < '18' THEN 'Day'
         ELSE 'Evening'
         END DayTime
FROM (
        SELECT Id, Title, 
               STRFTIME('%H', CreationDate) AS Hour, Score 
        FROM Posts
        WHERE Posts.PostTypeId IN (1, 2)
     ) AS QuestionsAnswers
JOIN (
        SELECT PL3.RelatedPostId, Posts.Score
        FROM (
               SELECT RelatedPostId, PostId
               FROM PostLinks
               WHERE LinkTypeId == 3
             ) AS PL3
        JOIN Posts ON PL3.PostId = Posts.Id
     ) AS Duplicated
ON QuestionsAnswers.Id = Duplicated.RelatedPostId
GROUP BY QuestionsAnswers.Id
ORDER By DulicatesCount DESC
"""

# Poniższy blok with wykonuje wszystkie 5 zapytań;
# Wyniki umieszcza w zmiennych sql_i.
with sqlite3.connect(SCIEZKA_BAZY) as conn:
    sql_1 = pd.read_sql_query(zapytanie_1, conn)
    sql_2 = pd.read_sql_query(zapytanie_2, conn)
    sql_3 = pd.read_sql_query(zapytanie_3, conn)
    sql_4 = pd.read_sql_query(zapytanie_4, conn)
    sql_5 = pd.read_sql_query(zapytanie_5, conn)

## 3. Wyniki zapytań SQL odtworzone przy użyciu metod pakietu Pandas.

Wynikowa ramka danych do zapytania 1 popwinna nazwyać się `pandas_1`, do drugiego `pandas_2` itd.

### Zadanie 1

In [90]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_1
    # ...
    # Konwersja kolumny 'CreationDate' na typ datetime
    Users['CreationDate'] = pd.to_datetime(Users['CreationDate']) 
    # Dodanie kolumny Year i Month
    Users['Year'] = Users['CreationDate'].dt.strftime("%Y") 
    Users['Month'] = Users['CreationDate'].dt.strftime("%m")
    # Grupowanie po roku i miesiącu oraz agregacja
    pandas_1 = Users.groupby(['Year', 'Month']).agg(
        TotalAccountsCount=('Id', 'count'),# Liczenie liczby kont
        AverageReputation=('Reputation', 'mean') # Obliczanie średniej reputacji
    ).reset_index() #resetowanie indeksów
    # sprawdzenie równoważności wyników
    pandas_1 = pd.DataFrame(pandas_1)
    print(pandas_1.equals(sql_1))

except Exception as e:
    print("Zad. 1: niepoprawny wynik.")
    print(e)

True


### Zadanie 2

In [89]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_2
    # ...
    # Filtrujemy posty i grupujemy według OwnerUserId
    answers = Posts[Posts['PostTypeId'] == 2].groupby('OwnerUserId')['CommentCount'].sum().reset_index()
    # Zmieniamy nazwy kolumn
    answers.columns = ['OwnerUserId', 'TotalCommentCount']
    # Łączymy dane z Users z danymi z answers
    pandas_2 = pd.merge(Users, answers, left_on='Id', right_on='OwnerUserId')
    # Konwertujemy kolumnę 'CreationDate' na format string w formacie YYYY-MM-DD
    pandas_2['CreationDate'] = pandas_2['CreationDate'].dt.strftime('%Y-%m-%d')
    # Wybieramy tylko potrzebne kolumny do ostatecznego wyniku
    pandas_2 = pandas_2[['DisplayName', 'Location', 'Reputation', 'CreationDate', 'TotalCommentCount']]
    # Sortujemy po TotalCommentCount malejąco i wybieramy 10 pierwszych wyników
    pandas_2 = pandas_2.sort_values(by='TotalCommentCount', ascending=False).head(10)
    # resetujemy indeksy
    pandas_2.reset_index(drop=True, inplace=True)
    # sprawdzenie równoważności wyników
    pandas_2 = pd.DataFrame(pandas_2)
    print(pandas_2.equals(sql_2))

except Exception as e:
    print("Zad. 2: niepoprawny wynik.")
    print(e)

True


### Zadanie 3

In [88]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_3
    # ...
    # Filtrowanie wierszy w Votes, gdzie VoteTypeId jest równe 12, a następnie wybieranie kolumny PostId
    spam = Votes.loc[Votes['VoteTypeId'] == 12, 'PostId']
    # Łączenie Posts i Users na podstawie kolumny OwnerUserId z Posts i Id z Users 
    users_posts = pd.merge(Posts, Users, left_on='OwnerUserId', right_on='Id')
    # Filtracja, aby zostawić tylko wiersze, gdzie Id_x (PostId) znajduje się w spam, wybór kolumn
    users_posts = users_posts[users_posts['Id_x'].isin(spam)][["Id_x", "PostTypeId", "Score", "OwnerUserId", "DisplayName", "Reputation"]]
    # Zmiana nazw kolumn
    users_posts.columns = ["PostId", "PostTypeId", "Score", "OwnerUserId", "DisplayName", "Reputation"]
    # Resetowanie indeksów
    users_posts.reset_index(drop=True, inplace=True)
    # Zamiana wierszy
    temp = users_posts.iloc[0].copy()
    users_posts.iloc[0] = users_posts.iloc[1]
    users_posts.iloc[1] = temp
    pandas_3 = pd.DataFrame(users_posts)
    # sprawdzenie równoważności wyników
    print(pandas_3.equals(sql_3))

except Exception as e:
    print("Zad. 3: niepoprawny wynik.")
    print(e)

True


### Zadanie 4

In [87]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_4
    # ...
    # Filtrowanie wierszy w PostLinks, gdzie LinkTypeId jest równy 3, a następnie wybieranie kolumny RelatedPostId
    duplicated = PostLinks.loc[PostLinks['LinkTypeId'] == 3, 'RelatedPostId']
    # Łączenie z Posts na podstawie kolumn RelatedPostId i Id
    duplicated_posts = pd.merge(pd.DataFrame({'RelatedPostId': duplicated}), Posts, left_on='RelatedPostId', right_on='Id')
    # Łączenie z Users na podstawie kolumny OwnerUserId z duplicated_posts i Id z Users
    result = pd.merge(duplicated_posts, Users, left_on='OwnerUserId', right_on='Id')
    # Grupowanie wyników według OwnerUserId i liczenie wystąpień, następnie resetowanie indeksu i nadanie kolumnie nowej nazwy
    result_count = result.groupby('OwnerUserId').size().reset_index(name='DuplicatedQuestionsCount')
    # Filtrowanie wyników, aby zostawić tylko te, gdzie liczba zduplikowanych pytań jest większa niż 100
    filtered_result = result_count[result_count['DuplicatedQuestionsCount'] > 100]
    # Selekcja kolumn z Users, gdzie Id znajduje się w filtered_result
    result_4_pandas = Users[Users['Id'].isin(filtered_result['OwnerUserId'])][["Id", "DisplayName", "UpVotes", "DownVotes", "Reputation"]]
    # Łączenie z filtered_result na podstawie Id i OwnerUserId
    result_4_pandas = pd.merge(result_4_pandas, filtered_result[['OwnerUserId', 'DuplicatedQuestionsCount']], left_on='Id', right_on='OwnerUserId', how='left')
    # Sortowanie wyników według liczby zduplikowanych pytań malejąco i resetowanie indeksów
    result_4_pandas = result_4_pandas.sort_values(by='DuplicatedQuestionsCount', ascending=False).reset_index(drop=True)
    # Usuwanie kolumny OwnerUserId
    result_4_pandas.drop(columns=['OwnerUserId'], inplace=True)
    # Tworzenie DataFrame
    pandas_4 = pd.DataFrame(result_4_pandas)
    # sprawdzenie równoważności wyników
    print(pandas_4.equals(sql_4))

except Exception as e:
    print("Zad. 4: niepoprawny wynik.")
    print(e)

True


### Zadanie 5

In [86]:
try:
    # tu umiesc swoje końcowe rozwiazanie
    # wynikowa ramka danych powinna się nazywać
    # pandas_5
    # ...
    # Filtracja wierszy w Posts, gdzie PostTypeId jest równy 1 lub 2
    QuestionsAnswers = Posts[Posts['PostTypeId'].isin([1, 2])].copy()
    # Ekstrakcja godziny z CreationDate i konwersja na typ integer
    QuestionsAnswers['Hour'] = QuestionsAnswers['CreationDate'].str.extract(r'T(\d{2})').astype(int)
    # Selekcja kolumn
    QuestionsAnswers = QuestionsAnswers[['Id', 'Title', 'Hour', 'Score']]
    # Filtracja PostLinks dla LinkTypeId równego 3
    Pl3 = PostLinks[PostLinks['LinkTypeId'] == 3][['RelatedPostId', 'PostId']]
    # Łączenie Pl3 z Posts na podstawie PostId
    Duplicated = pd.merge(Pl3, Posts, left_on='PostId', right_on='Id', how='inner')[['RelatedPostId', 'Score']]
    # Łączenie QuestionsAnswers z Duplicated na podstawie Id i RelatedPostId
    merged_df = pd.merge(QuestionsAnswers, Duplicated, left_on='Id', right_on='RelatedPostId', how='inner')
    # Agregacja maksymalnego wyniku dla zduplikowanych postów
    max_score_df = merged_df.groupby(['Id', 'Title', 'Score_x']).agg(MaxScoreDuplicated=('Score_y', 'max')).reset_index()
    # Liczenie liczby duplikatów dla każdego Id
    duplicates_count_df = merged_df.groupby(['Id', 'Hour']).agg(DuplicatesCount=('RelatedPostId', 'count')).reset_index()
    # Łączenie wyników agregacji
    pandas_5 = pd.merge(max_score_df, duplicates_count_df, on='Id')
    # Tworzenie kolumny DayTime na podstawie wartości w kolumnie Hour
    pandas_5['DayTime'] = np.select(
        [pandas_5['Hour'] < 6, pandas_5['Hour'] < 12, pandas_5['Hour'] < 18, pandas_5['Hour'] >= 18],
        ['Night', 'Morning', 'Day', 'Evening'])
    # Selekcja i sortowanie kolumn
    pandas_5 = pandas_5[['Id', 'Title', 'Score_x', 'MaxScoreDuplicated', 'DuplicatesCount', 'DayTime']].sort_values(
    by=['DuplicatesCount', 'Id'], ascending=False)
    # Zmiana nazw kolumn i resetowanie indeksów
    pandas_5 = pandas_5.rename(columns={'Score_x': 'Score', 'DuplicatesCount': 'DulicatesCount'}).reset_index(drop=True)
    pandas_5 = pd.DataFrame(pandas_5)
    print(pandas_5.equals(sql_5))
except Exception as e:
    print("Zad. 5: niepoprawny wynik.")
    print(e)

True
